In [1]:
from sympy import * 
from sympy.printing import print_ccode
init_printing(use_latex='mathjax')
from IPython.display import display_markdown

In [2]:
def display_vars(*args):
    if isinstance(args[0], str):
        args = (args,)
    display_markdown("$$" + ',\;'.join(name + ' = ' + latex(var) for name, var in args) + "$$", raw=True)

In [3]:
state  = Matrix(symbols('h, p, t, t_{os}', real=True))
predict = Matrix(symbols('\hat{h}, \hat{p}, \hat{t}, \hat{t}_{os}'))
predict_measure = Matrix(symbols('\hat{p}_m, \hat{t}_m'))

display_vars(("x", state),
             ("\hat{x}", predict),
             ("\hat{z}", predict_measure))

$$x = \left[\begin{matrix}h\\p\\t\\t_{os}\end{matrix}\right],\;\hat{x} = \left[\begin{matrix}\hat{h}\\\hat{p}\\\hat{t}\\\hat{t}_{os}\end{matrix}\right],\;\hat{z} = \left[\begin{matrix}\hat{p}_m\\\hat{t}_m\end{matrix}\right]$$

In [4]:
p0, g, m, r = symbols('p_0, g, m, r', real=True)

In [11]:
predict_fun = Matrix([state[0], p0 * exp(-state[0]*g*m/(r*state[2])), state[2], state[3]])

dt = Symbol("\Delta t", real=True)
transition_mat = exp(predict_fun.jacobian([state]) * dt)
display_vars(("f", predict_fun),
             ("F = e^{A \Delta t}", transition_mat))

$$f = \left[\begin{matrix}h\\p_{0} e^{- \frac{g h m}{r t}}\\t\\t_{os}\end{matrix}\right],\;F = e^{A \Delta t} = \left[\begin{matrix}e^{\Delta t} & 0 & 0 & 0\\- \frac{g m p_{0}}{r t} e^{\Delta t} e^{- \frac{g h m}{r t}} + \frac{g m p_{0}}{r t} e^{- \frac{g h m}{r t}} & 1 & \frac{g h m p_{0}}{r t^{2}} e^{\Delta t} e^{- \frac{g h m}{r t}} - \frac{g h m p_{0}}{r t^{2}} e^{- \frac{g h m}{r t}} & 0\\0 & 0 & e^{\Delta t} & 0\\0 & 0 & 0 & e^{\Delta t}\end{matrix}\right]$$

In [9]:
alpha = symbols('alpha', real=True)
predict_measure = Matrix(symbols('\hat{p}_m, \hat{t}_m'))
measure_fun = Matrix([predict[1] + alpha*(predict[2] + predict[3]),
                      predict[2] + predict[3]])

observation_mat = measure_fun.jacobian([predict])
display_vars("H", observation_mat)

$$H = \left[\begin{matrix}0 & 1 & \alpha & \alpha\\0 & 0 & 1 & 1\end{matrix}\right]$$

In [12]:
print(ccode(transition_mat, MatrixSymbol('out', 4, 4)).replace('\\Delta t', 'dt').replace('p_0', 'p0'))

out[0] = exp(dt);
out[1] = 0;
out[2] = 0;
out[3] = 0;
out[4] = -g*m*p0*exp(dt)*exp(-g*h*m/(r*t))/(r*t) + g*m*p0*exp(-g*h*m/(r*t))/(r*t);
out[5] = 1;
out[6] = g*h*m*p0*exp(dt)*exp(-g*h*m/(r*t))/(r*pow(t, 2)) - g*h*m*p0*exp(-g*h*m/(r*t))/(r*pow(t, 2));
out[7] = 0;
out[8] = 0;
out[9] = 0;
out[10] = exp(dt);
out[11] = 0;
out[12] = 0;
out[13] = 0;
out[14] = 0;
out[15] = exp(dt);
